##### L'objectif est de calculer des taux 

In [1]:
from datetime import date

from openfisca_france import init_country
from openfisca_france.model.base import *

ImportError: cannot import name init_country

## Adaptation pour faciliter l'usage de ce notebook

_Ce correctif permet de redéfinir plusieurs fois la même variable sans provoquer d'erreur._

In [2]:
import functools

from openfisca_core.formulas import make_reference_formula_decorator
from openfisca_france.entities import entity_class_by_symbol

reference_formula = make_reference_formula_decorator(entity_class_by_symbol = entity_class_by_symbol, update = True)
reference_input_variable = functools.partial(reference_input_variable, update = True)

# Variable avec formule

# Variable avec différentes formules en fonction de la date

# Système socio-fiscal

In [3]:
TaxBenefitSystem = init_country()
tax_benefit_system = TaxBenefitSystem()

# Simulation

In [4]:
# age = 60
# ages = [12, 37, 28]

# jean_eric = {
#     'name': u'Jean-Éric',
#     'age': 33,
# }
# aurelie = dict(
#     name = u'Aurélie',
#     age = 33,
# )

# personnes = [aurelie, jean_eric]

In [5]:
simulation = tax_benefit_system.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = 12000,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [6]:
simulation.calculate("salsuperbrut")

array([ 11180.56933594,   5180.56933594,      0.        ], dtype=float32)

In [7]:
simulation.calculate("revdisp")

array([ 20358.859375], dtype=float32)

# Réforme

In [8]:
from openfisca_core import reforms

In [9]:
Reform = reforms.make_reform(
    key = 'cotisations_revenu_de_base',
    name = u"Réforme des cotisations pour un Revenu de base",
    reference = tax_benefit_system,
    )

In [10]:
@Reform.formula
class cotisations_contributives(SimpleFormulaColumn):
    column = FloatCol
    entity_class = Individus
    label = u"Nouvelles cotisations contributives"


    def function(self, simulation, period):
        ags = simulation.calculate('ags', period)
        agff_tranche_a_employeur = simulation.calculate('agff_tranche_a_employeur', period)
        apec_employeur = simulation.calculate('apec_employeur', period)
        arrco_tranche_a_employeur = simulation.calculate('arrco_tranche_a_employeur', period)
        assedic_employeur = simulation.calculate('assedic_employeur', period)
        cotisation_exceptionnelle_temporaire_employeur = simulation.calculate('cotisation_exceptionnelle_temporaire_employeur', period)
        fonds_emploi_hospitalier = simulation.calculate('fonds_emploi_hospitalier', period)
        ircantec_employeur = simulation.calculate('ircantec_employeur', period)
        pension_civile_employeur = simulation.calculate('pension_civile_employeur', period)
        prevoyance_obligatoire_cadre = simulation.calculate('prevoyance_obligatoire_cadre', period)
        rafp_employeur = simulation.calculate('rafp_employeur', period)
        vieillesse_deplafonnee_employeur = simulation.calculate('vieillesse_deplafonnee_employeur', period)
        vieillesse_plafonnee_employeur = simulation.calculate('vieillesse_plafonnee_employeur', period)
        allocations_temporaires_invalidite = simulation.calculate('allocations_temporaires_invalidite', period)
        accident_du_travail = simulation.calculate('accident_du_travail', period)
        agff_tranche_a_employe = simulation.calculate('agff_tranche_a_employe', period)
        agirc_tranche_b_employe = simulation.calculate('agirc_tranche_b_employe', period)
        apec_employe = simulation.calculate('apec_employe', period)
        arrco_tranche_a_employe = simulation.calculate('arrco_tranche_a_employe', period)
        assedic_employe = simulation.calculate('assedic_employe', period)
        cotisation_exceptionnelle_temporaire_employe = simulation.calculate('cotisation_exceptionnelle_temporaire_employe', period)
        ircantec_employe = simulation.calculate('ircantec_employe', period)
        pension_civile_employe = simulation.calculate('pension_civile_employe', period)
        rafp_employe = simulation.calculate('rafp_employe', period)
        vieillesse_deplafonnee_employe = simulation.calculate('vieillesse_deplafonnee_employe', period)
        vieillesse_plafonnee_employe = simulation.calculate('vieillesse_plafonnee_employe', period)

        cotisations_contributives = (
            # cotisations patronales contributives dans le prive
            ags +
            agff_tranche_a_employeur +
            apec_employeur +
            arrco_tranche_a_employeur +
            assedic_employeur +
            cotisation_exceptionnelle_temporaire_employeur +
            prevoyance_obligatoire_cadre +  # TODO contributive ou pas
            vieillesse_deplafonnee_employeur +
            vieillesse_plafonnee_employeur +
            # cotisations patronales contributives dans le public
            fonds_emploi_hospitalier +
            ircantec_employeur +
            pension_civile_employeur +
            rafp_employeur +
            # anciennes cot patronales non-contributives classées ici comme contributives
            allocations_temporaires_invalidite +
            accident_du_travail +
            # anciennes cotisations salariales contributives dans le prive
            agff_tranche_a_employe +
            agirc_tranche_b_employe +
            apec_employe +
            arrco_tranche_a_employe +
            assedic_employe +
            cotisation_exceptionnelle_temporaire_employe +
            vieillesse_deplafonnee_employe +
            vieillesse_plafonnee_employe +
            # anciennes cotisations salariales contributives dans le public
            ircantec_employe +
            pension_civile_employe +
            rafp_employe
            )
        return period, cotisations_contributives
        

In [11]:
@Reform.formula
class nouv_salbrut(SimpleFormulaColumn):
    reference = tax_benefit_system.column_by_name['salaire_de_base']

# Le salaire brut se définit dans la réforme comme le salaire super-brut auquel
# on retranche les cotisations contributives 
    
    def function(self, simulation, period):
        period = period.start.period('month').offset('first-of')
        salsuperbrut = simulation.calculate('salsuperbrut', period)
        cotisations_contributives = simulation.calculate('cotisations_contributives', period)
        
        nouv_salbrut = (
            salsuperbrut -
            cotisations_contributives
            )
        return period, nouv_salbrut

In [12]:
@Reform.formula
class nouv_csg(SimpleFormulaColumn):
    reference = tax_benefit_system.column_by_name['csg_imposable_salaire']

# On applique une CSG unique à 22,5% qui finance toutes les prestations non-contributives 
    
    def function(self, simulation, period):
        period = period.start.period('month').offset('first-of')
        nouv_salbrut = simulation.calculate('nouv_salbrut', period)
        
        nouv_csg = (
            -0.225 * nouv_salbrut
            )
        return period, nouv_csg

In [13]:
@Reform.formula
class salnet(SimpleFormulaColumn):
    reference = tax_benefit_system.column_by_name['salaire_net']

# On retire la nouvelle CSG (pas celle qui finance le RDB) pour trouver le nouveau salaire net
    
    def function(self, simulation, period):
        period = period.start.period('month').offset('first-of')
        nouv_salbrut = simulation.calculate('nouv_salbrut', period)
        nouv_csg = simulation.calculate('nouv_csg', period)
        
        salnet = (
            nouv_salbrut +
            nouv_csg
            )
        return period, salnet

In [14]:
@Reform.formula
class sal(SimpleFormulaColumn):
    reference = tax_benefit_system.column_by_name['salaire_de_base']

# Nous sommes partis du nouveau salaire net et par rapport au salaire imposable actuel,
# nous avons supprimé : les heures sup, la déductibilité de CSG
    
    
    def function(self, simulation, period):
        period = period
        salnet = simulation.calculate('salnet', period)
        primes_fonction_publique = simulation.calculate('primes_fonction_publique', period)
        indemnite_residence = simulation.calculate('indemnite_residence', period)
        supp_familial_traitement = simulation.calculate('supp_familial_traitement', period)
        rev_microsocial_declarant1 = simulation.calculate('rev_microsocial_declarant1', period)

        return period, (
            salnet +
            primes_fonction_publique +
            indemnite_residence +
            supp_familial_traitement +
            hsup +
            rev_microsocial_declarant1
            )

In [15]:
reform = Reform()

In [16]:
reform_simulation = reform.new_scenario().init_single_entity(
    period = 2014,
    parent1 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = 12000,
        ),
    parent2 = dict(
        birth = date(1980, 1, 1),
        salaire_de_base = 6000,
        ),
    enfants = [
        dict(
            birth = date(2014, 1, 1),
            ),
        ],
    ).new_simulation(debug = True)

In [17]:
reform_simulation.calculate('salsuperbrut')

array([ 11180.56933594,   5180.56933594,      0.        ], dtype=float32)

In [18]:
reform_simulation.calculate('cotisations_contributives')

ERROR:openfisca_core.formulas:An error occurred while calling formula cotisations_contributives@individus<2014> in module __main__


KeyError: 'agff_tranche_a_employeur'

In [ ]:
reform_simulation.calculate('nouv_salbrut')

In [ ]:
reform_simulation.calculate('salbrut')

In [ ]:
reform_simulation.calculate('salnet')